In [ ]:
#install what needs to be installed setting up intance here

In [ ]:
#Google Drive mount here
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
#set up file location here
import sys
sys.path.append('/content/gdrive/LLMHallucination')

In [ ]:
# import libraries
from Util import load_data_JSON, load_model
import pandas as pd
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [ ]:
#load the data
general_df,prompt_g,correct_g,gtp4_g,Info_g = load_data_JSON('input/general_data.json','general')
qa_df,prompt_q,correct_q,gtp4_q,Info_q = load_data_JSON('input/qa.json','qa')
sum_df,prompt_s,correct_s,gtp4_s,Info_s = load_data_JSON('input/summarization_data.json','sum')

In [ ]:
access = input('Access code?')

In [ ]:
#setup of model
tokenizer, model = load_model("meta-llama/Llama-2-7b-chat-hf",access)

In [ ]:

def listOfTuples(l1, l2):
    return list(map(lambda x, y:(x,y), l1, l2)) 


In [ ]:
#run of the prompts 
out_q =[]
qp = listOfTuples(prompt_q,Info_q)
for p in qp:
    # Tokenize the prompt
    inputs = tokenizer(p[0], return_tensors="pt").to("cuda")

    # Generate the response
    with torch.no_grad():
        outputs = model.generate(
            inputs['input_ids'],
            max_length=50,  # Adjust this based on your needs
            num_beams=5,  # Optional: Use beam search to improve generation quality
            early_stopping=True)

    # Decode the output
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    prompt_qa =  "You are a huallucination detector. You MUST determine if the provided answer contains hallucination or not for the question based on the provided Knowledge. The answer you provided MUST be \"Yes\" or \"No\""+ "\n\n#Knowledge#: " + p[1]+"\n\n#Question#: " + p[0] + "\n#Answer#: " + response + "\n#Your Judgement#:"
    inputs = tokenizer(prompt_qa, return_tensors="pt").to("cuda")

    # Generate the response
    with torch.no_grad():
        outputs = model.generate(
            inputs['input_ids'],
            max_length=50,  # Adjust this based on your needs
            num_beams=5,  # Optional: Use beam search to improve generation quality
            early_stopping=True)

    # Decode the output
    response2 = tokenizer.decode(outputs[0], skip_special_tokens=True)
    out_q.append(response,response2)

In [ ]:
#run of the prompts 
out_s =[]
for p in prompt_s:
    # Tokenize the prompt
    inputs = tokenizer("Summarize this document: "+p, return_tensors="pt").to("cuda")

    # Generate the response
    with torch.no_grad():
        outputs = model.generate(
            inputs['input_ids'],
            max_length=500,  # Adjust this based on your needs
            num_beams=5,  # Optional: Use beam search to improve generation quality
            early_stopping=True)

    # Decode the output
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    prompt_su =  "You are a summary judge. You MUST determine if the provided summary contains non-factual or hallucinated information. The answer you give MUST be \"Yes\" or \"No\""+ "\n\n#Document#: " + p+"\n\n#Question#: "+ "\n#Summary#: " + response + "\n#Your Judgement#:"
    inputs = tokenizer(prompt_su, return_tensors="pt").to("cuda")

    # Generate the response
    with torch.no_grad():
        outputs = model.generate(
            inputs['input_ids'],
            max_length=50,  # Adjust this based on your needs
            num_beams=5,  # Optional: Use beam search to improve generation quality
            early_stopping=True)

    # Decode the output
    response2 = tokenizer.decode(outputs[0], skip_special_tokens=True)
    out_q.append(response,response2)

In [ ]:
#write